# Wrapping a template library

A template library is a library where there are only template classes that can be instantiated.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider various *C++* template classes instantiations during the `Parse` step.
It is therefore required to install the `pyclanglite` `parser`.

In [1]:
%%bash
conda install -y -q python-clanglite -c statiskit

Fetching package metadata .........
Solving package specifications: ..........

Package plan for installation in environment /home/main/miniconda:

The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-env-2.5.2            |           py27_0          26 KB

The following packages will be SUPERCEDED by a higher-priority channel:

    conda:     4.1.11-py27_1 conda-forge --> 4.1.11-py27_0
    conda-env: 2.5.2-py27_0  conda-forge --> 2.5.2-py27_0 



Using Anaconda API: https://api.anaconda.org


The **Standard Template Library (STL)** library is a *C++* library that provides a set of common *C++* template classes such as containers and associative arrays.
These classes can be used with any built-in or user-defined type that supports some elementary operations (e.g. copying, assignment).
It is divided in four components called algorithms, containers, functional and iterators.
**STL** containers (e.g. `std::vector`, `std::set`) are used in many *C++* libraries.
In such a case, it does not seem relevant that every wrapped *C++* library contains wrappers for usual **STL** containers (e.g. `std::vector< double >`, `std::set< int >`).
We therefore proposed *Python* bindings for sequence containers (i.e. `pair`, `array`, `vector`, `deque`, `forward_list` and `list` of the `std` namespace) and associative containers (`set`, `multiset`, `map`, `multimap`, `unordered_set`, `unordered_multiset`, `unordered_map` and `unordered_multimap` of the `std` namespace).
These template instantiations are done for *C++* fundamental types (`bool`, `signed char`, `unsigned char`, `char`, `wchar_t`, `int` (with sign modifiers `signed` and `signed` combined or not with size modifiers `short`, `long` and `long long`), `float`, `double`, `long double`) and strings (`string`, `wstring` of the `std` namespace).
For ordered associative containers both `std::less` and `std::greater` comparators are used.
We here only illustrate the procedure on the `std::vector` template class.
For the complete procedure refers to the `AutoWIG.py` file situed at the root of the **PySTL** [repository](https://github.com/StatisKit/PySTL).

The **PySTL** GitHub repository must be cloned into the **PySTL** directory.

In [2]:
%%bash
git clone https://github.com/StatisKit/PySTL.git

Cloning into 'PySTL'...


This repository already has wrappers, we therefore need to remove them.

In [3]:
from path import path
srcdir = path('PySTL')/'src'/'py'
for wrapper in srcdir.walkfiles('*.cpp'):
    wrapper.unlink()
for wrapper in srcdir.walkfiles('*.h'):
    wrapper.unlink()
wrapper = srcdir/'stl'/'vector'/'_vector.py'
if wrapper.exists():
    wrapper.unlink()

Then, to install and compile the *C++* library we use available **Conda** recipes.

In [4]:
%%bash --err error
conda build -q PySTL/conda/libpystl -c statiskit
conda install -y -q libpystl --use-local -c statiskit

Removing old build environment
BUILD START: libpystl-1.0.0-0
    (actual version deferred until further download or env creation)
Fetching package metadata ...........
Solving package specifications: ..........
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/cpp/array.h" as "/home/main/miniconda/envs/_build/include/pystl/array.h"
Install file: "build/cpp/deque.h" as "/home/main/miniconda/envs/_build/include/pystl/deque.h"
Install file: "build/cpp/forward_list.h" as "/home/main/miniconda/envs/_build/include/pystl/forward_list.h"
Install file: "build/cpp/map.h" as "/home/main/miniconda/envs/_build/include/pystl/map.h"
Install file: "build/cpp/multimap.h" as "/home/main/miniconda/envs/_build/include/pystl/multimap.h"
Install file: "build/cpp/multiset.h" as "/home/main/miniconda/envs/_build/include/pystl/multiset.h"
Install file: "build/cpp/set.h" as "/home/main/miniconda/envs/_build/include/pystl/set.h"
Install fil

As presented below, in order to wrap a template library, the user needs to write headers containing aliases for desired template class instantiations.

In [5]:
%%bash
pygmentize PySTL/src/cpp/vector.h

#include <vector>
#include <string>

namespace pystl
{
    namespace vector
    {
        typedef std::vector< bool > Bool;
        typedef std::vector< char > Char;
        typedef std::vector< signed char > SignedChar;
        typedef std::vector< unsigned char > UnsignedChar;
        typedef std::vector< int > Int;
        typedef std::vector< short > Short;
        typedef std::vector< short int > ShortInt;
        typedef std::vector< long > Long;
        typedef std::vector< long int > LongInt;
        typedef std::vector< long long > LongLong;
        typedef std::vector< long long int > LongLongInt;
        typedef std::vector< signed > Signed;
        typedef std::vector< signed  int > SignedInt;
        typedef std::vector< signed short > SignedShort;
        typedef std::vector< signed short int > SignedShortInt;
        typedef std::vector< signed long > SignedLong;
        typedef std::vector< signed long int > SignedLongInt;
        typedef std::vector< signed long long >

Once these preliminaries done, we can proceed to the actual generation of wrappers for the **PySTL** library.
For this, we import **AutoWIG** and create an empty Abstract Semantic Graph (ASG).

In [6]:
import autowig
asg = autowig.AbstractSemanticGraph()

We need then to install the *C++* headers. This is done using the `cpp` target in **SCons**.

In [7]:
%%bash
scons cpp -C PySTL

bash: line 1: scons: command not found


Once the headers habe been installed in the system, we parse headers with relevant compilation flags.

In [8]:
%%time
import sys
autowig.parser.plugin = 'pyclanglite'
asg = autowig.parser(asg,
                     [sys.prefix + '/include/pystl/vector.h'],
                     flags = ['-x', 'c++', '-std=c++11',
                              '-I' + sys.prefix + '/include'],
                     bootstrap = 1,
                     silent = True)

/home/main/miniconda/lib/python2.7/site-packages/clanglite/autowig_parser.py:818: Warning: 
  warnings.warn('', Warning)


CPU times: user 5.63 s, sys: 52 ms, total: 5.68 s
Wall time: 5.76 s


Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is thus suitable.

In [9]:
%%time
autowig.controller.plugin = 'default'
asg = autowig.controller(asg)

CPU times: user 3.57 s, sys: 8 ms, total: 3.58 s
Wall time: 3.57 s


In order to wrap the chosen instations of the `std::vector` template class, we need to select the `boost_python_internal` `generator` implementation.

In [10]:
%%time
autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg,
                  module = './PySTL/src/py/_vector.cpp',
                  prefix = '_vector_wrapper_',
                  decorator = './PySTL/src/py/stl/vector/_vector.py',
                  closure = True)

CPU times: user 3.65 s, sys: 4 ms, total: 3.65 s
Wall time: 3.65 s


The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [11]:
%%time
wrappers.write()

CPU times: user 9.23 s, sys: 24 ms, total: 9.26 s
Wall time: 9.25 s


Here is an example of the generated wrappers.
We here present the wrappers for the `std::vector< int >` class.

In [12]:
%%bash
pygmentize PySTL/src/py/_vector_wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp

#include "_vector.h"


namespace autowig
{
    void method_decorator_7debf7c14b9b59bda0df7817656d79e8(class ::std::vector< int, class ::std::allocator< int > > & instance, unsigned long int  param_in_0, int  & param_out)
    { instance.operator[](param_in_0) = param_out; }
    void method_decorator_bb1e0852f2ca56c094260a03787426c7(class ::std::vector< int, class ::std::allocator< int > > & instance, unsigned long int  param_in_0, int  & param_out)
    { instance.at(param_in_0) = param_out; }
    void method_decorator_7ec1ac72b0b05f3a9707175bcd5da0bd(class ::std::vector< int, class ::std::allocator< int > > & instance, int  & param_out)
    { instance.front() = param_out; }
    void method_decorator_ed1cf37568ed54cbbd326e6ccbe5f27d(class ::std::vector< int, class ::std::allocator< int > > & instance, int  & param_out)
    { instance.back() = param_out; }
}


void _vector_wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3()
{

    std::string name_a5e4e9231d6351ccb0e06756b389f0af = boost::python::

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.

In [13]:
%%bash
conda build PySTL/conda/python-stl-vector --use-local -c statiskit 
conda install -y python-stl-vector --use-local -c statiskit

Removing old build environment
BUILD START: python-stl-vector-1.0.0-py27_0
    (actual version deferred until further download or env creation)
Fetching package metadata ...........
Solving package specifications: ..........
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/cpp/vector.h" as "/home/main/miniconda/envs/_build/include/pystl/vector.h"
g++ -o build/py/_vector.h.gch -x c++-header -c -fPIC -std=c++0x -ftemplate-depth-100 -DBOOST_PYTHON_DYNAMIC_LIB -I/home/main/miniconda/envs/_build/include -I/home/main/miniconda/envs/_build/include/python2.7 build/py/_vector.h
g++ -o build/py/_vector_wrapper_628e6cb7cb2e5fed9cbdad40da9cebd7.os -c -std=c++0x -ftemplate-depth-100 -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -I/home/main/miniconda/envs/_build/include -I/home/main/miniconda/envs/_build/include/python2.7 build/py/_vector_wrapper_628e6cb7cb2e5fed9cbdad40da9cebd7.cpp
g++ -o build/py/_vector_wrapper_448c20257e485acda59dc593

Using Anaconda API: https://api.anaconda.org

latest version is 2.0.0. Run

conda update -n root conda-build

to get the latest version.

+ source activate /home/main/miniconda/envs/_build
++ [[ -n 4.3.11(1)-release ]]
++ _SCRIPT_LOCATION=/home/main/miniconda/envs/_build/bin/activate
++ _SHELL=bash
+++ dirname /home/main/miniconda/envs/_build/bin/activate
++ _CONDA_DIR=/home/main/miniconda/envs/_build/bin
++ '[' 1 -gt 1 ']'
++ case "$(uname -s)" in
+++ uname -s
++ EXT=
++ [[ -n 4.3.11(1)-release ]]
+++ basename /home/main/miniconda/conda-bld/work/conda_build.sh
++ [[ conda_build.sh == \a\c\t\i\v\a\t\e ]]
++ '[' 1 -eq 0 ']'
++ args=/home/main/miniconda/envs/_build
++ /home/main/miniconda/envs/_build/bin/conda ..checkenv bash /home/main/miniconda/envs/_build
++ ((  0 != 0  ))
++ source /home/main/miniconda/envs/_build/bin/deactivate
+++ [[ -n 4.3.11(1)-release ]]
+++ _SCRIPT_LOCATION=/home/main/miniconda/envs/_build/bin/deactivate
+++ _SHELL=bash
++++ dirname /home/main/miniconda/envs/_b

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [ ]:
from stl import vector
v = vector.Int()
v.push_back(-1)
v.push_back(0)
v.push_back(1)
v

In [ ]:
list(v)

In [ ]:
v[0]

In [ ]:
v[0] = -2
v[0]

In [ ]:
vector.Int([0, 1])